In [ ]:
import torch
torch.cuda.empty_cache()
!python -m vllm.entrypoints.openai.api_server --model ./model/Meta-Llama-3.1-8B-Instruct-lora  --served-model-name Meta-Llama-3.1-8B-Instruct-lora --max-model-len=4096

/usr/local/lib/python3.10/site-packages/_distutils_hack/__init__.py:55: UserWarning: Reliance on distutils from stdlib is deprecated. Users must rely on setuptools to provide the distutils module. Avoid importing distutils or import setuptools first, and avoid setting SETUPTOOLS_USE_DISTUTILS=stdlib. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
2024-09-06 20:13:28.649428: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-06 20:13:28.660219: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-06 20:13:28.673787: E external/local_xla/xla/stream_executor/cuda/c

In [2]:
from llama_index.core import SimpleDirectoryReader
from ReadLoad import read_jsonl, write_jsonl
import json
from llama_index.core import Settings
from llama_index.core import Document
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

In [3]:
def read_json(path):
    with open(path, 'r') as file:
        data = json.load(file)
    return data

ana = read_json('data/external_data/ana.json')
qwen_data = read_json('data/external_data/qwen_train_data_1697.json')

In [4]:
#模型下载
from modelscope import snapshot_download
model_dir = snapshot_download('xorbits/bge-small-zh-v1.5', cache_dir="./model")

2024-09-06 20:03:19,343 - modelscope - WARNING - Model revision not specified, use revision: v0.0.2


In [5]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# 假设 model_dir 是你下载模型的本地路径
model_path = model_dir

# 使用本地模型路径创建 HuggingFaceEmbedding 实例
embedding = HuggingFaceEmbedding(
    model_name=model_path,  # 使用本地模型路径
    cache_folder=model_dir,  # 指定缓存目录
    embed_batch_size=128
)

Settings.embed_model = embedding

In [7]:
rag_data = ana + qwen_data
qa_dict = {}
questions = []
for i, data in enumerate(rag_data):
    question = data['instruction'].split("题目如下：")[1]
    answer = data['output']
    id_ = i
    data['answer'] = answer
    data['question'] = question
    data['rag_id'] = id_
    qa_dict[id_] = answer

In [8]:
documents = [Document(text=t['question'], metadata={"rag_id": t['rag_id']}) for t in rag_data]
vector_index = VectorStoreIndex.from_documents(documents)

In [9]:
import torch
torch.cuda.empty_cache()

In [ ]:
def get_few_shot(query, k=3):
    vector_retriever = vector_index.as_retriever(similarity_top_k=k)
    nodes = vector_retriever.retrieve(query)
    string = ''
    for node in nodes:
        question = node.text
        answer_id = node.metadata['rag_id']
        answer = qa_dict[answer_id]
        qa = '\n### 回答:\n'.join([question,answer])
        string += qa + '\n\n'
    return string